In [1]:
# Importing the requisite packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### Which state receives the most spending on political ads?

In [2]:
# Screening the dataset and retaining the necessary variables
df=pd.read_csv("google-political-ads-geo-spend.csv")

In [3]:
df=df[df["Country"]=="IN"]

In [4]:
df=df[["Country_Subdivision_Primary","Spend_INR"]]

In [5]:
# Exporting the file to create a choropleth map on QGIS
df.to_csv("ad_spend_by_state.csv", index=False)

#### Analysis of google ad spending during the 2019 General Elections

In [6]:
# Importing the dataset that contains the total cost and total number of ads by advertiser - only using it for the names and ids of advertisers
indian_advertisers=pd.read_csv("google-political-ads-advertiser-stats.csv")

In [7]:
# Screening out data for other countries
indian_advertisers=indian_advertisers[indian_advertisers["Regions"]=="IN"]

In [8]:
# Retaining the Advertiser IDs and Names for India - not using spending data, as it covers non-election periods
indian_advertisers=indian_advertisers[["Advertiser_ID","Advertiser_Name"]]

In [9]:
# Importing a dataset that contains data on expenditure per week 
weekly_spend=pd.read_csv("google-political-ads-advertiser-weekly-spend.csv")

In [10]:
# Only retaining Indian advertisers
weekly_spend=weekly_spend.merge(indian_advertisers, how="right")

In [11]:
# Screening out advertisements that were not in the election period
weekly_spend["Week_Start_Date"]=pd.to_datetime(weekly_spend["Week_Start_Date"], format="%Y-%m-%d")

In [12]:
model_code=pd.to_datetime("10-03-2019", format="%d-%m-%Y")

# Drooping advertisements that were made after the election results were declared on 23-May
upper_date_limit=pd.to_datetime("23-05-2019", format="%d-%m-%Y")
weekly_spend=weekly_spend[(weekly_spend["Week_Start_Date"]<upper_date_limit)]

In [13]:
# Dropping unnecessary variables
weekly_spend=weekly_spend[["Advertiser_ID", "Advertiser_Name", "Week_Start_Date" ,"Spend_INR"]]

In [14]:
# Exporting a file with the names of advertisers to a csv to manually map advertisers to NDA/UPA/Other
party_map=pd.DataFrame(weekly_spend["Advertiser_Name"].value_counts()).reset_index().drop(columns=["Advertiser_Name"]).rename(columns={"index":"Advertiser_Name"})
party_map.to_csv("advertiser_names.csv", index=False)

In [15]:
# Importing the mapped csv
party_map=pd.read_csv("advertiser_names_completed.csv")

In [16]:
# Merging in alliance, and party information into the weekly spend data
weekly_spend=weekly_spend.merge(party_map, on="Advertiser_Name", how="left")

In [17]:
# There is one advertiser for two parties = AAP and TDP - Assigning individual party names after manually checking the videos 
## released in the specified week
weekly_spend.loc[20,"Party"]="TDP"
weekly_spend.loc[21,"Party"]="TDP"
weekly_spend.loc[22,"Party"]="AAP"
weekly_spend.loc[22,"Policy_violations"]=0

In [18]:
spend_share=pd.DataFrame(weekly_spend.groupby(by="Party").Spend_INR.sum()).reset_index()

In [19]:
# Total spend on election ads, and share of spending by political party
spend_share["total_spend"]=spend_share["Spend_INR"].sum()

In [20]:
spend_share["party_pct"]=spend_share["Spend_INR"]/spend_share["total_spend"]*100

In [21]:
# Excluding 16 advertisers whose ads have been removed due to policy violations and they cannot be mapped to a political party.
# Excluding 2 advertisers who ads are apolitical

weekly_spend=weekly_spend[(weekly_spend["Alliance"]!="REMOVED") & (weekly_spend["Alliance"]!="NONE")]

#### Top parties by spending on Google Ads 
- BJP spent over 4 times the amount that INC spent
- TDP, DMK spent almost twice more than INC
- Top 6 parties

In [22]:
total_spend=pd.DataFrame(weekly_spend.groupby(by="Party").Spend_INR.sum().sort_values(ascending=False)).reset_index()

total_spend.to_csv("spending.csv", index=False)

#### Top alliances by spending on Google Ads
- NDA spent 75% more on Google ads than UPA
- In fact, other parties - not affiliated to either NDA or UPA spent more than the UPA

In [23]:
weekly_spend.groupby(by="Alliance").Spend_INR.sum().sort_values(ascending=False)

Alliance
NDA      124097750
OTHER     78678750
UPA       70794000
Name: Spend_INR, dtype: int64

#### Policy violations
- Google removed a total of 389 political ads posted during the elections
- 187 of these cannot be linked to a political party (the videos have been removed, and we cannot link the advertiser to a political party)
- INC had the most number of policy violations - 101, followed by TDP, YSRCP, BJP - 28

In [24]:
print(f"Google removed a total of {party_map.Policy_violations.sum():.0f} political ads posted during the elections.")

Google removed a total of 389 political ads posted during the elections.


In [25]:
violations_party=pd.DataFrame(party_map.groupby(by="Party").Policy_violations.sum().sort_values(ascending=False)).reset_index()

In [26]:
violations_party["Party"]=violations_party["Party"].str.replace("TDP, AAP", "TDP")

In [27]:
# Excluding unidentified advertisers
violations_party=violations_party[(violations_party["Party"]!="REMOVED")&(violations_party["Party"]!="NONE")]

In [28]:
violations_party=violations_party.groupby(by="Party").sum().reset_index().sort_values(by="Policy_violations", ascending=False)

In [29]:
violations_party

,Party,Policy_violations
4,INC,101.0
7,TDP,35.0
8,YSRCP,31.0
2,BJP,28.0
3,DMK,6.0
1,AITC,1.0
0,AIADMK,0.0
5,JSP,0.0
6,MNM,0.0


#### Ad-level characteristics

In [30]:
# Importing data on all ads and their creative stats
creative_stats=pd.read_csv("google-political-ads-creative-stats.csv")

C:\Users\Anuja\AppData\Local\Programs\Python\Python37-32\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [31]:
# Only retaining ads posted in India
creative_stats=creative_stats[creative_stats["Regions"]=="IN"]

In [32]:
# Excluding unnecessary variables
creative_stats=creative_stats[["Ad_ID","Ad_Type","Advertiser_ID","Advertiser_Name","Date_Range_Start","Date_Range_End","Num_of_Days","Impressions"]]

In [33]:
# Excluding ads that were posted outside the election period
creative_stats["Date_Range_Start"]=pd.to_datetime(creative_stats["Date_Range_Start"], format="%Y-%m-%d")
creative_stats["Date_Range_End"]=pd.to_datetime(creative_stats["Date_Range_End"], format="%Y-%m-%d")

In [34]:
creative_stats=creative_stats[(creative_stats["Date_Range_Start"]<upper_date_limit)]

In [35]:
# Creating a master list of ads that were posted during the election period, and that are assigned a political party
ads_master=creative_stats.merge(party_map, how="left")

In [36]:
# Total number of ads
ads_master.groupby(by="Party").Ad_ID.count()

Party
AIADMK        37
AITC         315
BJP         9897
DMK          266
INC          615
JSP            2
MNM            2
NONE           2
REMOVED      185
TDP          192
TDP, AAP     124
YSRCP        291
Name: Ad_ID, dtype: int64

In [37]:
ads_master=ads_master[(ads_master["Party"]!="REMOVED") & (ads_master["Party"]!="NONE")]

In [38]:
# Merging in targeting data
target=pd.read_csv("google-political-ads-campaign-targeting.csv")

In [39]:
# Excluding advertisers that are not from India
target=target.merge(indian_advertisers, how="right")

In [40]:
# The dataset is organised by campaign - reshaping it to an ad per row
target["Ads_List_2"]=target["Ads_List"].str.split(",")

In [41]:
target.reset_index(inplace=True)

In [42]:
target=target.rename(columns={"index":"uid"})

In [43]:
target_ads=pd.DataFrame(target["Ads_List_2"].values.tolist())

In [44]:
target_ads.reset_index(inplace=True)

In [45]:
target_ads=target_ads.rename(columns={"index":"uid"})

In [46]:
target_ads=target_ads.melt(id_vars="uid", value_vars=list(range(0,159))).dropna().sort_values(by="uid")

In [47]:
target_ads=target_ads.drop(columns=["variable"])

In [48]:
target_ads=target_ads.rename(columns={"value":"Ad_ID"})

In [49]:
target=target.merge(target_ads, on="uid")

In [50]:
target=target.drop(columns=["uid","Campaign_ID","Ads_List_2","Ads_List"])
# This dataset now has all ads that were targeted for India (even those outside the election period) 

In [51]:
# Coding the target variables as binary
target["Target_Age"]=np.where(target["Age_Targeting"]!="Not targeted", 1,0)
target["Target_Gender"]=np.where(target["Gender_Targeting"]!="Not targeted", 1,0)
target["Geo_Included"]=np.where(target["Geo_Targeting_Included"]!="Not targeted", 1,0)
target["Geo_Excluded"]=np.where(target["Geo_Targeting_Excluded"]!="Not targeted",1,0)

In [52]:
target["Target_Geo"]=target["Geo_Included"]+target["Geo_Excluded"]

In [53]:
target["Target_Geo"]=np.where(target["Target_Geo"]>0,1,0)

In [54]:
## Retaining only the necessary variables
target.columns
target=target[["Advertiser_ID","Advertiser_Name","Ad_ID","Target_Age", "Target_Gender","Target_Geo"]]

In [55]:
# Merging it into the ads_master dataset, only retaining ads from the master list
ads_master=ads_master.merge(target, how="left")

In [56]:
ads_master["Target_Age"]=ads_master["Target_Age"].fillna(0)
ads_master["Target_Gender"]=ads_master["Target_Gender"].fillna(0)
ads_master["Target_Geo"]=ads_master["Target_Geo"].fillna(0)

In [57]:
ads_master["Whether_targetted"]=ads_master["Target_Age"]+ads_master["Target_Gender"]+ads_master["Target_Geo"]

In [58]:
ads_master["Whether_targetted"]=np.where(ads_master["Whether_targetted"]>0,1,0)

In [59]:
# TDP and AAP had the same advertiser, so these ads are clubbed together as TDP, AAP - Assigning individual party names after
## manually checking the videos

tdp_aap=ads_master[ads_master["Party"]=="TDP, AAP"]

def party_assignment(date):
    date1=pd.to_datetime("30-04-2019", format="%d-%m-%Y")
    date2=pd.to_datetime("01-05-2019", format="%d-%m-%Y")
    if (date==date1) or (date==date2):
        return "AAP"
    else:
        return "TDP"

tdp_aap["Party"]=tdp_aap["Date_Range_Start"].apply(party_assignment)

C:\Users\Anuja\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [60]:
ads_master_1=ads_master[ads_master["Party"]!="TDP, AAP"]

In [61]:
ads_master=ads_master_1.append(tdp_aap)

In [62]:
# Generating a dataset with the following indicators - total number of ads, total number of ads targeted by each - age, gender 
# and geo, type of ad, and median number of days per ad. 

In [63]:
party_list=ads_master["Party"].unique()

In [64]:
# Ads targetted

target_data=[]

for party in party_list:
    df=ads_master[ads_master["Party"]==party]
    target_info={}
    target_info["Party"]=party
    try:
         target_info["Ads_target"]=df.groupby(by="Whether_targetted").Ad_ID.count()[1]
    except:
        target_info["Ads_target"]=0
    target_data.append(target_info)

all_targeted=pd.DataFrame(target_data)

In [65]:
# Age targetted

target_age_data=[]

for party in party_list:
    df=ads_master[ads_master["Party"]==party]
    target_age={}
    target_age["Party"]=party
    try:
         target_age["Age_target"]=df.groupby(by="Target_Age").Ad_ID.count()[1]
    except:
        target_age["Age_target"]=0
    target_age_data.append(target_age)

age_targeted=pd.DataFrame(target_age_data)

In [66]:
target_gender_data=[]

for party in party_list:
    df=ads_master[ads_master["Party"]==party]
    target_gender={}
    target_gender["Party"]=party
    try:
         target_gender["Gender_target"]=df.groupby(by="Target_Gender").Ad_ID.count()[1]
    except:
        target_gender["Gender_target"]=0
    target_gender_data.append(target_gender)

gender_targeted=pd.DataFrame(target_gender_data)

In [67]:
# Geo targetted

target_geo_data=[]

for party in party_list:
    df=ads_master[ads_master["Party"]==party]
    target_geo={}
    target_geo["Party"]=party
    try:
         target_geo["Geo_target"]=df.groupby(by="Target_Geo").Ad_ID.count()[1]
    except:
        target_geo["Geo_target"]=0
    target_geo_data.append(target_geo)

geo_targeted=pd.DataFrame(target_geo_data)

In [68]:
# Total_Ads

all_data=[]

for party in party_list:
    df=ads_master[ads_master["Party"]==party]
    all_ads={}
    all_ads["Party"]=party
    all_ads["Total_ads"]=df.Ad_ID.count()
    all_data.append(all_ads)

all_ads_df=pd.DataFrame(all_data)

In [69]:
# Ad_type

ad_type_data=[]

for party in party_list:
    df=ads_master[ads_master["Party"]==party]
    ad_type={}
    ad_type["Type"]=df["Ad_Type"].value_counts().index.tolist()
    ad_type["Number_of_ads"]=df["Ad_Type"].value_counts().values.tolist()
    ad_type["Party"]=party
    ad_type_data.append(ad_type)

In [70]:
ad_type_df=pd.DataFrame()
for party in ad_type_data:
    party_df=pd.DataFrame(party)
    ad_type_df=ad_type_df.append(party_df)

In [71]:
ad_type_df=ad_type_df.pivot(index="Party", columns="Type", values="Number_of_ads").reset_index().fillna(0)

In [72]:
# Impressions

impressions_data=[]

for party in party_list:
    df=ads_master[ads_master["Party"]==party]
    impressions={}
    impressions["Type"]=df["Impressions"].value_counts().index.tolist()
    impressions["Number_of_ads"]=df["Impressions"].value_counts().values.tolist()
    impressions["Party"]=party
    impressions_data.append(impressions)

In [73]:
impressions_df=pd.DataFrame()
for party_impression in impressions_data:
    each_party=pd.DataFrame(party_impression)
    impressions_df=impressions_df.append(each_party)

In [74]:
impressions_df=impressions_df.pivot(index="Party", columns="Type", values="Number_of_ads").reset_index().fillna(0)

In [75]:
# Median number of days per ad
median_days=pd.DataFrame(ads_master.groupby(by="Party").Num_of_Days.median()).reset_index()

In [76]:
# Average period per party - diff b/w earliest start date for an ad and latest end date for an ad
earliest_date=pd.DataFrame(ads_master.groupby(by="Party").Date_Range_Start.min()).reset_index()

In [77]:
latest_date=pd.DataFrame(ads_master.groupby(by="Party").Date_Range_End.max()).reset_index()

In [78]:
duration=earliest_date.merge(latest_date)

In [79]:
duration["Avg_days"]=duration["Date_Range_End"]-duration["Date_Range_Start"]

In [80]:
duration=duration[["Party","Avg_days"]]

In [81]:
# Merging all of these datasets
ad_char=all_ads_df.merge(ad_type_df, how="left").merge(all_targeted, how="left").merge(age_targeted, how="left").merge(gender_targeted, how="left").merge(geo_targeted, how="left").merge(impressions_df, how="left").merge(median_days, how="left").merge(duration, how="left")

In [82]:
# Calculating percentages

ad_char["Pct_image"]=(ad_char["Image"]/ad_char["Total_ads"])*100
ad_char["Pct_text"]=(ad_char["Text"]/ad_char["Total_ads"])*100
ad_char["Pct_video"]=(ad_char["Video"]/ad_char["Total_ads"])*100


ad_char["Pct_targeted"]=(ad_char["Ads_target"]/ad_char["Total_ads"])*100

ad_char["Subset_Age_pct"]=(ad_char["Age_target"]/ad_char["Ads_target"])*100
ad_char["Subset_Gender_pct"]=(ad_char["Gender_target"]/ad_char["Ads_target"])*100
ad_char["Subset_Geo_pct"]=(ad_char["Geo_target"]/ad_char["Ads_target"])*100


ad_char["Pct_100k-1M"]=ad_char["100k-1M"]/ad_char["Total_ads"]*100
ad_char["Pct_10k-100k_pct"]=ad_char["10k-100k"]/ad_char["Total_ads"]*100
ad_char["Pct_1M-10M"]=ad_char["1M-10M"]/ad_char["Total_ads"]*100
ad_char["Pct_> 10M"]=ad_char["> 10M"]/ad_char["Total_ads"]*100
ad_char["Pct_≤ 10k"]=ad_char["≤ 10k"]/ad_char["Total_ads"]*100

In [83]:
ad_char_2=ad_char.drop(columns=["Image", "Text", "Video", "Ads_target", "Age_target", "Gender_target", "Geo_target", "100k-1M", "10k-100k","1M-10M","> 10M", "≤ 10k"])

In [84]:
# Merging in the spending and violations datasets generated above
all_data=ad_char_2.merge(total_spend, how="outer").merge(violations_party, how="outer").fillna(0)

C:\Users\Anuja\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Passing integers to fillna is deprecated, will raise a TypeError in a future version.  To retain the old behavior, pass pd.Timedelta(seconds=n) instead.
  


In [85]:
all_data

,Party,Total_ads,Num_of_Days,Avg_days,Pct_image,Pct_text,Pct_video,Pct_targeted,Subset_Age_pct,Subset_Gender_pct,Subset_Geo_pct,Pct_100k-1M,Pct_10k-100k_pct,Pct_1M-10M,Pct_> 10M,Pct_≤ 10k,Spend_INR,Policy_violations
0,JSP,2,4,11 days,0.000000,0.000000,100.000000,100.000000,100.000000,100.000000,100.000000,0.000000,50.000000,0.000000,0.000000,50.000000,27000,0.0
1,YSRCP,291,5,24 days,76.975945,0.000000,23.024055,30.584192,73.033708,82.022472,91.011236,31.615120,23.367698,19.243986,4.467354,21.305842,22698750,31.0
2,INC,615,4,97 days,68.780488,2.276423,28.943089,24.878049,79.084967,77.124183,97.385621,34.471545,34.146341,9.268293,2.113821,20.000000,30732000,101.0
3,TDP,310,5,19 days,68.387097,0.000000,31.612903,28.387097,62.500000,62.500000,100.000000,23.225806,16.451613,22.258065,5.161290,32.903226,45808250,35.0
4,DMK,266,3,9 days,73.308271,0.000000,26.691729,13.909774,48.648649,51.351351,100.000000,14.661654,16.541353,23.684211,0.375940,44.736842,40062000,6.0
5,AITC,315,17,42 days,48.571429,0.000000,51.428571,52.063492,98.170732,98.170732,100.000000,41.587302,26.666667,17.142857,1.269841,13.333333,10058500,1.0
6,BJP,9920,4,91 days,98.356855,0.020161,1.622984,11.602823,4.952215,4.691573,100.000000,7.993952,16.854839,2.731855,0.423387,71.995968,122359750,28.0
7,MNM,2,1,0 days,0.000000,0.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,0.000000,0.000000,0.000000,0.000000,80000,0.0
8,AIADMK,37,3,6 days,59.459459,0.000000,40.540541,51.351351,78.947368,78.947368,100.000000,59.459459,16.216216,21.621622,0.000000,2.702703,1738000,0.0
9,AAP,9,2,1 days,0.000000,0.000000,100.000000,11.111111,0.000000,0.000000,100.000000,0.000000,33.333333,0.000000,0.000000,66.666667,6250,0.0


#### The End!